In [7]:
import os
import scipy.io as sio
import numpy as np
from PIL import Image
import cv2
import copy


In [8]:
cur_dir = os.getcwd()
# in_dir = os.path.join(cur_dir, "gt")
in_dir = os.path.join(cur_dir, "rendered")
# out_dir = os.path.join(cur_dir, "gt_tf")
out_dir = os.path.join(cur_dir, "rendered_tf")

In [46]:
for filename in os.listdir(in_dir):
    filepath = os.path.join(in_dir, filename)
    im = Image.open(filepath)
    in_depth = np.array(im)
    out_depth = 1 - 1 / (in_depth + 1)
    tiff_name = filename[:-5] + ".tiff"
    tiff_path = os.path.join(out_dir, tiff_name)
    tiff_path = tiff_path.replace("distance_mean", "depth")
    out_im = Image.fromarray(out_depth)
    out_im.save(tiff_path)

In [13]:
near = 2.0
far = 6.0

gt = False

for filename in os.listdir(in_dir):
    filepath = os.path.join(in_dir, filename)
    if gt:
        filepath = os.path.join(in_dir, filename)
        im = cv2.imread(filepath, cv2.IMREAD_UNCHANGED) / 8192
        im = im[..., 0]
        tiff_name = filename[:-4] + ".png"
    else:
        im = Image.open(filepath)
        tiff_name = filename[:-5] + ".png"
    in_depth = np.array(im)
    out_depth = np.clip((in_depth - near) / (far - near), 0, 1) * 256.0
    tiff_path = os.path.join(out_dir, tiff_name)
    tiff_path = tiff_path.replace("distance_mean", "depth")
    out_im = Image.fromarray(out_depth)
    out_im = out_im.convert('RGB')
    copy.deepcopy(out_im).save(tiff_path, "PNG")

In [14]:
for filename in os.listdir(in_dir):
    if filename.endswith(".mat"):
        filepath = os.path.join(in_dir, filename)
        data = sio.loadmat(filepath)

        for value in data.values():
            if isinstance(value, np.ndarray):
                # mat_array = 1 - (1 / value)
                mat_array = 1 - 1 / (value + 1)
                break

        tiff_name = filename[:-4] + ".tiff"
        tiff_path = os.path.join(out_dir, tiff_name)
        Image.fromarray(np.nan_to_num(mat_array).astype(np.float32)).save(tiff_path, 'TIFF')